<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Export-calibration-stream" data-toc-modified-id="Export-calibration-stream-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Export calibration stream</a></span></li><li><span><a href="#Get-the-image-and-the-data" data-toc-modified-id="Get-the-image-and-the-data-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Get the image and the data</a></span></li><li><span><a href="#Set-the-scale" data-toc-modified-id="Set-the-scale-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Set the scale</a></span></li><li><span><a href="#Process-calibration-image" data-toc-modified-id="Process-calibration-image-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Process calibration image</a></span></li><li><span><a href="#Fit-the-model" data-toc-modified-id="Fit-the-model-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Fit the model</a></span></li></ul></div>

In [1]:
%matplotlib qt
import f3st
from f3st import deep_reload
deep_reload(f3st)
from f3st.calibration import *
from f3st.deposit_model import *
from f3st.utils import 

import matplotlib.pyplot as plt

### Export calibration stream

In [3]:
calib_strm_savepath = 'testing'
# add create safe savename and the wide calibration
export_spot_calibration(calib_strm_savepath, start_time=1, end_time=15, grid=[5, 3])

### Get the image and the data

In [215]:
img_path = 'testing\calib1_001.tif'
data_path = 'testing\calib25-11-19_data.txt'
observing_angle = 45

data = np.loadtxt(data_path, delimiter='\t', skiprows=1)
img = read_image(img_path)
plt.imshow(img)

### Set the scale

In [216]:
tracker = select_scale(img)

82, 21
402, 22


In [222]:
scale_bar_size = 2 # in um

scale_bar_px = np.abs(tracker.scale_markers[1] - tracker.scale_markers[0])
ppn = scale_bar_px / scale_bar_size
print('Pixels per nanometer: ', ppn)

Pixels per nanometer:  159.76774193548385


### Process calibration image

In [218]:
# remove the bar and threshold the image
img_nobar = remove_bottom_bar(img)
img_thresh = threshold_image(img_nobar, thresh=None)
# try_all_threshold(img_smooth)
plt.imshow(img_thresh)

In [219]:
# label image regions
label_image, image_label_overlay = get_labelled_image(img_thresh)
labels = filter_small_labels(label_image)
image_label_overlay = label2rgb(label_image, image=img_thresh, bg_label=0)
plt.imshow(image_label_overlay)

In [223]:
# get lengths of the structures
lengths_px = get_lengths_px(label_image)
lengths_perspective = np.sort(lengths_px) / ppn
lengths_true = lengths_perspective / np.sin(np.deg2rad(observing_angle))
dwell_times = data[-lengths.size:, 0]

### Fit the model

In [243]:
# fit the deposit model and get the parameters
deposit_model, popt, _ = DDModel.fit_calibration(dwell_times, lengths_true)

t = np.linspace(0, np.max(dwell_times), 1000)
plt.plot(t, deposit_model(t, *popt), color='tab:orange', linewidth=2)
plt.scatter(dwell_times, lengths_true)
plt.xlabel('t [s]')
plt.ylabel(r'Length [$\mu$m]');

GR:  0.20040780644948566
k:  1.124924446842553


In [260]:
addressable_pixels=[65536, 56576]
grid=[5, 3]
xstep, ystep = np.array(addressable_pixels) * 0.8 / np.array(grid)
x = np.arange(grid[0])*xstep
y = np.arange(grid[1])*ystep


# np.grid

In [261]:
xygrid = np.meshgrid(x, y)

In [267]:
np.array(xygrid).reshape(2, 15)

array([20971.52,     0.  ])

In [269]:
t = np.arange(10)
np.random.shuffle(t)
t

array([9, 5, 6, 3, 2, 4, 1, 7, 0, 8])

In [251]:
65536*0.8/5

10485.76

In [550]:

deep_reload(f3st)
from f3st import *
from f3st.calibration.vertical_structures import *

120000.0
